# Traits, Type Parameters, Generics and Type Constraints

In this lecture, we will cover the concepts of traits, type parameters, generics, and constraints
on type parameters in Scala. This material is covered in chapters 12 and 19 of the Oderseky et al
book on Scala. 

### Outline

- Inheritance: super calls.
- Final keyword in scala (read on your own).
- Traits: what are traits?
- Examples with traits.
- Things we can and cannot do with a trait: 
  - Cannot take in class parameters
  - If we add members to a trait, the class that extends a trait has to implement those members
  - Traits can inherit from a base class but that limits the kind of classes that can make use of the traits
- Talk about type parameters in functions

## Inheritance: Super Calls

Note that thus far, we have talked about how inheritance works. We also noted the concept of dynamic dispatch.
Suppose we defined a function `foo` inside the base class A and override that function `foo` inside the derived class B.


  

In [91]:
class A {
    def foo() = println("From A: Base Class")
}

class B extends A {
    override def foo() = {println("From B: Derived Class")}
}


defined class A
defined class B

Suppose we created an object `b` of type B and upcast it to an object `a` of type A. What happens if we call `a.foo()`? Clearly, dynamic dispatch means that in runtime scala recognizes that object `a` was originally created as a B and upcast. It therefore calls the `foo` method defined in B. The following code illustrates this clearly.

In [92]:
def callFoo(a: A) = a.foo()

val b = new B()
callFoo(b) // Upcast implicitly by scala since callFoo takes in an argument of type A

From B: Derived Class


defined function callFoo
b: B = $sess.cmd90Wrapper$Helper$B@786c6389

Suppose our goal is to call the super class method from the B, we can use a super call. This simply
uses `super` as a reference to the _immediate_ super class of a given class. For instance, inside an instance
of object B, saying `super` refers to an object of type A.

In [93]:
class A { 
    def foo() =  println("From A: whatever")
}

class B extends A {
    override def foo() = { super.foo(); println("From B: nonsense") }
    
    def bar() = {super.foo()}
}

val b = new B()
b.foo()

From A: whatever
From B: nonsense


defined class A
defined class B
b: wrapper.wrapper.B = $sess.cmd92Wrapper$Helper$B@6fd5f46f

## Traits

Traits are an important mechanism for code reuse in scala. They allow us to define functionality that can be 
exported across multiple objects in the overall hierarchy. A trait is almost like an abstract class or an interface. It can define its own members and methods.

In [94]:
trait Philosophical {
    def philosophize: Unit = println(s"I take up space, therefore I am !")
}

trait Green {
    def color: String = "green"
}


defined trait Philosophical
defined trait Green

Traits can be _applied_ or _mixed in_ to classes so that the methods of the trait and the members are part of the
object itself.

In [95]:
class Frog(val name: String) extends Philosophical {
    def getName: String = name
    override def toString: String= "Frog " + name
}

defined class Frog

In [23]:
val f = new Frog("Freddie")
val k = new Frog("Kermit")
f.philosophize
k.philosophize

I take up space, therefore I am !
I take up space, therefore I am !


f: Frog = Frog Freddie
k: Frog = Frog Kermit

You cannot directly create a trait since traints are considered abstract. Well, to be honest there is a way to do this (if you are interested). **Chapter 20** of the scala textbook describes this and you may want to take a look.

In [5]:
val t = new Philosophical()

cmd5.sc:1: trait Philosophical is abstract; cannot be instantiated
val t = new Philosophical()
        ^

: 

Traits are useful as modifications that we can apply and stack on top of each other. 

### Pattern 1: Create useful functionality that can be applied to many classes.

Here is a slightly contrived example. Suppose I have a good pretty printer that can be used
to print debug messages but can be turned off if we ever needed to. We could build a trait
that can be applied to objects which need debug messages. The flag debug can be set to false
and the debug messages will stop printing to the standard output.

In [9]:
trait DebugPrinter {
    val debug: Boolean
    def debugPrint(s: String) = if (debug) println("Debug: " + s)
}

class A extends DebugPrinter {
    val debug = true // Turn off if you do not want debugging
    def foo() = {
        debugPrint("Called Foo")
        // Code goes here.
        debugPrint("Done with Foo")
    }
}

defined trait DebugPrinter
defined class A

Notice how the trait has a field `debug` in it. It is important therefore that any object that seeks to mixin a trait will need to define debug and set it appropriately during construction.

## Pattern 2: Define an interface that can be implemented by various objects.

Here is an example. We define an a comparison relation over objects of type T. The name of the trait
is `Ordering[T]`. The object that extends this trait implements the `compare` operator. Once that is
done: the trait itself implements a bunch of useful operators like <, > , ==, <= that can all be inherited.

In [13]:
trait Ordering[T]  { 
    //The user defines a compare operator that 
    // returns == 0 if two objects are equal
    //  > 0 if this > t
    //  < 0 if this < t
    def compare(t: T): Int 
    
    // The trait can automatically define operators for us to use
    def < (t:T): Boolean = this.compare(t) < 0
    
    def > (t:T): Boolean = this.compare(t) > 0
    
    def <= (t: T): Boolean = this.compare(t) <= 0
    
    def >= (t :T): Boolean = this.compare(t) >= 0
    
    def ==(t:T): Boolean = this.compare(t) == 0
    
    /* def sortList(l: List[T]): List[T] = { // Implement a generic sort algorithm using < operator
        
    }*/
}

defined trait Ordering

In [14]:
class A (val x: Int, val y: Int) extends Ordering[A] {
    def compare(a: A) = {
        if (this.x - a.x != 0) 
            this.x - a.x 
        else  // the x values are equal, just compare y values
            this.y - a.y
    }
}

defined class A

In [19]:
val a = new A(10, 15)
val b = new A(12, 18)
val c = new A(9, 19)

println(a < b)
println(a < c)
println(a >= c)
println(a <= a)

true
false
true
true


a: A = $sess.cmd13Wrapper$Helper$A@6c6e8f24
b: A = $sess.cmd13Wrapper$Helper$A@5e7a4c7f
c: A = $sess.cmd13Wrapper$Helper$A@5276ee65

Note that traits cannot take in parameters in their definition

In [19]:

trait Color (val color: String) { 
    def getColor: String = color
}

: 

The correct way to define this is to add a field called color to the trait. 

In [20]:

trait Color {
    val color: String
    def getColor: String = color
}


defined trait Color

## Multiple Traits Applied to an Object

Whereas a class in scala cannot extend from multiple base classes, it can extend from a single base class and a bunch of traits that can be mixed in. Note that we use `extends` keyword for the first class/trait that we inherit from/mixin and then a bunch of other traits can be mixed-in using the `with` keyword.

In [29]:
abstract class Animal 
// We can use extends keyword for the first inheritance from an object or for the first trait mixed in.
// The subsequent traits are mixed in using the with keyword.
class GreenFrog(n: String) extends Animal with Philosophical with Color { 
    // Without this line we will get a syntax error
    val color="green"
    override def philosophize: Unit = println(s"It ain't easy being $color")
}

defined class Animal
defined class GreenFrog

In [30]:
val f = new GreenFrog("froyo")
f.philosophize

It ain't easy being green


f: GreenFrog = $sess.cmd28Wrapper$Helper$GreenFrog@1ca99765

In [96]:
class MultiColorFrog(n: String, val color: String) extends Animal with Philosophical with Color 

defined class MultiColorFrog

Traits themselves can inherit from a class or another trait. 
If a trait inherits from a class A, it poses a restriction that the trait can only be mixed into classes
that themselves inherit from A.

In [28]:
abstract class A 

abstract class C 

trait Friendly extends A 

class D extends A with Friendly // OK

class E extends D // OK

class F extends E with Friendly // OK since F inherits from E, which inherits from D and in turn inherits from A.


defined class A
defined class C
defined trait Friendly
defined class D
defined class E
defined class F

In [28]:
class B extends C with Friendly  // NOT OK C does not inherit from A

cmd28.sc:1: illegal inheritance; superclass C
 is not a subclass of the superclass A
 of the mixin trait Friendly
class B extends C with Friendly  // NOT OK C does not inherit from A
                       ^

: 

## Type Parameters

We have thus far seen definitions like `Ordering[T]` that had a type parameter `[T]` attached to it. What is the meaning of the `[T]` parameter?

Here is an example of a class C that has a parameer `[T]`.

In [33]:
abstract class C[T] {
    val t: T
    def transform(arg: T): String = arg.toString
}

defined class C

`T` stands for a type that can be any type: an object defined by us or a predefined object such as `Int`, `String`, `List[String]` and so on.

We can instantiate `T` in many ways.

In [41]:
class D extends C[Int] {
    val t = 42
    override def transform(arg: Int): String = {s"Transform to $arg"}
}

class E[T](val t: T) extends C[T] {
    override def transform(arg: T): String = s"Better way to transform ${arg.toString}"
}

val e = new E("hello") //No need to say E[String] Why?

val f = new E(List("Hello", "World", "Whatever")) // Scala type infers that f has type E[List[String]]


class F (val x: String) 

val g: E[F] = new E(new F("Hello"))

defined class D
defined class E
e: wrapper.wrapper.E[String] = $sess.cmd40Wrapper$Helper$E@7e676bd0
f: wrapper.wrapper.E[List[String]] = $sess.cmd40Wrapper$Helper$E@4384fb39
defined class F
g: wrapper.wrapper.E[wrapper.wrapper.F] = $sess.cmd40Wrapper$Helper$E@22025e17

## Constraints on Type Parameters

We often want to assume that when we have a type parameter T, that T has a certain function foo defined inside it so that we could call the foo method of any object of type t. In general, it is not enough to pass any type parameter but to have some things that we would like to assume about the type parameter. How do we achieve that?

In [41]:
def callFoo[T](arg: T): Unit = arg.foo(25)

cmd41.sc:1: value foo is not a member of type parameter T
def callFoo[T](arg: T): Unit = arg.foo()
                                   ^

: 

The attempt above failed. Scala being strongly typed wants an assurance that T has a method called foo that takes in an `Int` and returns `Unit`. How do we say this?
- Define an abstract class or a trait called A.
- Say that T must be a derived class of A. To do so we say `T <: A` the `<:` means `is a subtype of` (or is a derived class of).

In [44]:
trait A {
    def foo(i: Int): Unit
}

def callFoo[T <: A ](arg: T ): Unit = {
    arg.foo(42)
}

defined trait A
defined function callFoo

In [46]:
class C extends A {
    override def foo(i: Int): Unit = println(s"phoo - $i")
}

val c = new C()
callFoo(c)

phoo - 42


defined class C
c: wrapper.wrapper.C = $sess.cmd45Wrapper$Helper$C@9940a9c

In [50]:
class A  {
    val t = 25
}

abstract class B extends A {
    def foo(): Int
}

def callFoo[T <: B] (t: T) = t.foo() // Scala infers return type is Int

class C extends B {
    override def foo() = 42
}

callFoo(new C())

defined class A
defined class B
defined function callFoo
defined class C
res49_4: Int = 42

Notice that callFoo has the constraint that `T` can only be a subclass of `B`. For instance, we cannot try to use a class of type A.

In [50]:
def wrapper(a: A) = callFoo(a)

cmd50.sc:1: inferred type arguments [cmd50Wrapper.this.cmd49.wrapper.A] do not conform to method callFoo's type parameter bounds [T <: cmd50Wrapper.this.cmd49.wrapper.B]
def wrapper(a: A) = callFoo(a)
                    ^cmd50.sc:1: type mismatch;
 found   : cmd50Wrapper.this.cmd49.wrapper.A
 required: T
def wrapper(a: A) = callFoo(a)
                            ^

: 

Sometimes we may imagine an opposite situation where we do not want foo to be called on objects that are greater than some object in the type hierarchy.

The following exposition is inspired from this wonderful blog post: https://apiumhub.com/tech-blog-barcelona/scala-type-bounds/

In [87]:

trait Thing {
    val name: String 
}
trait Vehicle extends Thing

class Car (val name: String) extends Vehicle {
    def getName: String = s"car + $name"
}

class Bus (val name: String) extends Vehicle

// The class parameter name in Sedan must be the same name in car. 
// Therefore, you need to say override
// Read chapters 4, 10 of the Scala book
class Sedan (override val name: String) extends Car(name) {
    override def getName: String = s"sedan + $name"
}


abstract class BigCar(override val name: String) extends Sedan(name) {
    override def getName: String = s"bigcar + $name"
}

class Jeep (override val name: String) extends BigCar(name) {
    override def getName: String = s"jeep + $name"
}

class MotorCycle(override val name: String) extends Vehicle 

class Tomato extends Thing { 
    override val name = "Tomato "
}

defined trait Thing
defined trait Vehicle
defined class Car
defined class Bus
defined class Sedan
defined class BigCar
defined class Jeep
defined class MotorCycle
defined class Tomato

Drawing a tree diagram helps one visualize the class hierarchy that the above definitions have created.
Let us now define a function that will park a vehicle.

In [75]:
def parkVehicle[A] (a: A) = println(s"Parked vehicle: ${a.name}")

cmd75.sc:1: value name is not a member of type parameter A
def parkVehicle[A] (a: A) = println(s"Parked vehicle: ${a.name}")
                                                          ^

: 

Oops: The type parameter A must have a field called name. To ensure that Scala is happy, we will add a constraint.

In [76]:
def parkVehicle[A <: Thing](a: A) = println(s"Parked vehicle: ${a.name}")

defined function parkVehicle

In [77]:
parkVehicle(new Sedan("Toyota Yaris"))

Parked vehicle: Toyota Yaris


Ooops: But I can now park a tomato

In [78]:
parkVehicle(new Tomato)

Parked vehicle: Tomato 


I can also park a bus

In [79]:
parkVehicle(new Bus("School Bus"))

Parked vehicle: School Bus


Let us fix it by adding an appropriate constraint.

In [80]:
def parkVehicle[A <: Car] (a: A) = println(s"Parked vehicle: ${a.name}")

defined function parkVehicle

In [80]:
parkVehicle(new Bus("School Bus"))

cmd80.sc:1: inferred type arguments [cmd80Wrapper.this.cmd74.wrapper.Bus] do not conform to method parkVehicle's type parameter bounds [A <: cmd80Wrapper.this.cmd79.cmd74.wrapper.Car]
val res80 = parkVehicle(new Bus("School Bus"))
            ^cmd80.sc:1: type mismatch;
 found   : cmd80Wrapper.this.cmd74.wrapper.Bus
 required: A
val res80 = parkVehicle(new Bus("School Bus"))
                        ^

: 

In [80]:
parkVehicle(new Tomato)

cmd80.sc:1: inferred type arguments [cmd80Wrapper.this.cmd74.wrapper.Tomato] do not conform to method parkVehicle's type parameter bounds [A <: cmd80Wrapper.this.cmd79.cmd74.wrapper.Car]
val res80 = parkVehicle(new Tomato)
            ^cmd80.sc:1: type mismatch;
 found   : cmd80Wrapper.this.cmd74.wrapper.Tomato
 required: A
val res80 = parkVehicle(new Tomato)
                        ^

: 

However, we do not want to be able to park Jeeps since they are too big for parking spot. We can right an annotation like so using `>:` to say that `A` must be a super class of BigCar in the class hierarchy.

In [88]:
def parkCompactVehicle[A >: BigCar <: Car] (a: A) = println(s"Parked vehicle: ${a.getName} in a compact spot")

defined function parkCompactVehicle

In [88]:
parkCompactVehicle[Jeep](new Jeep("Volvo"))


cmd88.sc:1: type arguments [cmd88Wrapper.this.cmd86.wrapper.Jeep] do not conform to method parkCompactVehicle's type parameter bounds [A >: cmd88Wrapper.this.cmd87.cmd86.wrapper.BigCar <: cmd88Wrapper.this.cmd87.cmd86.wrapper.Car]
val res88 = parkCompactVehicle[Jeep](new Jeep("Volvo"))
                              ^

: 

In [89]:
parkCompactVehicle[Sedan](new Jeep("Volvo")) // Can you tell me why this works? 

Parked vehicle: jeep + Volvo in a compact spot


In [90]:
val s: Sedan = new Sedan("Toyota Yaris")
parkCompactVehicle[Sedan](s)

Parked vehicle: sedan + Toyota Yaris in a compact spot


s: Sedan = $sess.cmd86Wrapper$Helper$Sedan@710abf94